**Introduction**
Generating the Extra Columns wiith earth Engine API

In [1]:
import geopandas as gpd
from shapely.geometry import shape , mapping
import pandas as pd
import time
import os
import numpy as np
from shapely import wkt
import geemap
import ee
import json
# from google.colab import auth , drive
# drive.mount('/content/drive') Only required if you are using google colab

In [3]:

df = gpd.read_file("../input/consolidated_reforestation_projects.geojson")

df.info()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/io/file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/io/file.py:387: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 801222 entries, 0 to 801221
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   project_id_reported       746726 non-null  object  
 1   trees_planted_reported    13193 non-null   float64 
 2   country                   795834 non-null  object  
 3   description_reported      2548 non-null    object  
 4   planting_date_reported    751711 non-null  object  
 5   geometry_reported         801222 non-null  object  
 6   survival_rate_reported    1269 non-null    float64 
 7   site_sqkm                 801222 non-null  float64 
 8   site_id_reported          757652 non-null  object  
 9   host_name                 801222 non-null  object  
 10  url                       801222 non-null  object  
 11  species_count_reported    0 non-null       object  
 12  species_planted_reported  270 non-null     object  
 13  created_site_ids     

In [5]:
df["geometry"]

0         POLYGON ((-5561391.868 -1038670.909, -5561494....
1         POLYGON Z ((-4839336.675 -2554053.909 0.000, -...
2         POLYGON Z ((-4838167.931 -2552670.718 0.000, -...
3         POLYGON Z ((-4838872.807 -2554348.003 0.000, -...
4         POLYGON Z ((-224756.486 911808.834 0.000, -224...
                                ...                        
801217    POLYGON ((-6438930.160 -3609277.645, -6438930....
801218    POLYGON ((-6105893.149 -3908169.191, -6105368....
801219    POLYGON ((-6105696.076 -3907505.595, -6105716....
801220    POLYGON ((-6129944.978 -3906375.852, -6129776....
801221    POLYGON ((-6002151.683 -3813346.665, -6001723....
Name: geometry, Length: 801222, dtype: geometry

In [6]:

ee.Initialize(project='spring-idiom-398208')


##TREE COVER COLUMNS; "tree_cover_area_2000,tree_cover_area_2005 tree_cover_area_2010,tree_cover_area_2015 AND tree_cover_area_2020

In [11]:
import geopandas as gpd
import json
import ee
import os
import pandas as pd
import time
import geemap
from shapely.geometry import shape
from shapely.validation import make_valid



#  cleaning  and loading GeoJSON safely
def safe_json_load(row):
    if isinstance(row, str):
        try:
            geojson_dict = json.loads(row.replace("'", '"'))
            geom = shape(geojson_dict)
            return make_valid(geom).buffer(0)  
        except json.JSONDecodeError:
            return None
    elif isinstance(row, dict):
        geom = shape(row)
        return make_valid(geom).buffer(0)  
    else:
        return row

# Converting geometry column to valid 2D geometries
df['geometry'] = df['geometry'].apply(safe_json_load)
df = df.set_geometry('geometry')
df = df.to_crs(epsg=4326)  # Ensuring CRS is WGS 84 (EPSG:4326)

# Dropping any rows with invalid geometries
df_valid = df.dropna(subset=['geometry'])

gdf = gpd.GeoDataFrame(df_valid, geometry='geometry')

# Spliting data into chunks for processing as earth engine has data loadinging limitataions
chunk_size = 1000
chunks = [gdf[i:i + chunk_size] for i in range(0, gdf.shape[0], chunk_size)]

# Defining the  Earth Engine  GLAD landcover images
landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020').updateMask(landmask)

# Listing all class codes for GLAD land cover
classCodes = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
              125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]

# Masking land cover image to only include class codes of interest for tree cover
maskedLandCover = landCover.remap(classCodes, classCodes, 255)

#  Calculating area for each class list in the codes
def calculate_area(feature):
    total_area = ee.Number(0)
    def calculate_class_area(class_code, total):
        class_area = maskedLandCover.eq(ee.Number(class_code)) \
            .multiply(ee.Image.pixelArea()) \
            .reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9
            ).get('remapped')

        area_value = ee.Number(class_area).divide(1e6)  # Converting the area to km^2
        return ee.Number(total).add(area_value)

    total_area = ee.List(classCodes).iterate(calculate_class_area, total_area)
    return feature.set('tree_cover_area_2020', total_area)


output_csv_path = '../input/Reforestation_Data/merged_land_cover_results.csv'

#  combining the  results DataFrame
if os.path.exists(output_csv_path):
    combined_df = pd.read_csv(output_csv_path)
else:
    combined_df = pd.DataFrame()

# Looping through chunks and process each in GEE
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__
    
    try:
        # Converting to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the area calculation function over the FeatureCollection
    area_results_chunk = fc_chunk.map(calculate_area)

    # Exporting each chunk to Google Drive as CSV
    task = ee.batch.Export.table.toDrive(
        collection=area_results_chunk,
        description=f'asia_chunk_{i + 1}',
        folder='Reforestation_Data',
        fileNamePrefix=f'temp_chunk_{i + 1}_new_2020_land_cover',
        fileFormat='CSV'
    )
    task.start()

    print(f"Exporting chunk {i + 1}...")

    while task.active():
        print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
        time.sleep(30)

    print(f'Chunk {i + 1} export done.')

    # Loading the exported CSV for this chunk
    temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_new_2020_land_cover.csv'
    temp_chunk_df = pd.read_csv(temp_chunk_path)

    # Appending to combined DataFrame
    combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)

    # Saving combined results to the output CSV
    combined_df.to_csv(output_csv_path, index=False)

    # Removing the temporary file
    if os.path.exists(temp_chunk_path):
        os.remove(temp_chunk_path)
        print(f"Deleted {temp_chunk_path}")

print("All chunks processed, combined results saved to:", output_csv_path)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


Processing chunk 1/802
Could not convert the geojson to ee.Geometry()
Error converting chunk 1 to Earth Engine FeatureCollection: Invalid GeoJSON geometry.
Processing chunk 2/802
Could not convert the geojson to ee.Geometry()
Error converting chunk 2 to Earth Engine FeatureCollection: Invalid GeoJSON geometry.
Processing chunk 3/802
Exporting chunk 3...
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODRTR) of chunk 3.
Polling for task (id: KDCB2E2AL2B52SFF73OODR

##CHANGE COLUMNS EXTRACTION ("permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees")

In [ ]:


# Defining the Earth Engine GLAD landcover images
landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC').updateMask(landmask)

# Class codes and names of interest
classCodes = [208, 240, 248, 245]
classesOfInterest = ["permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees"]

# Masking land cover image to only include class codes of interest
maskedLandCover = landCover.remap(classCodes, classCodes, 255)

# Calculating area for each class
def calculate_area(feature):
    total_area = ee.Number(0)
    def calculate_class_area(class_code, total):
        class_area = maskedLandCover.eq(ee.Number(class_code)) \
            .multiply(ee.Image.pixelArea()) \
            .reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9
            ).get('remapped')

        area_value = ee.Number(class_area).divide(1e6)  # Converting area to km^2
        return ee.Number(total).add(area_value)

    total_area = ee.List(classCodes).iterate(calculate_class_area, total_area)
    return feature.set('land_cover_area_2020', total_area)


output_csv_path = '../input/Reforestation_Data/extra_land_cover_results.csv'

# Combining the results DataFrame
if os.path.exists(output_csv_path):
    combined_df = pd.read_csv(output_csv_path)
else:
    combined_df = pd.DataFrame()

# Looping through chunks and processing each in GEE
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__
    
    try:
        # Converting to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the area calculation function over the FeatureCollection
    area_results_chunk = fc_chunk.map(calculate_area)

    # Exporting each chunk to Google Drive as CSV
    task = ee.batch.Export.table.toDrive(
        collection=area_results_chunk,
        description=f'asia_chunk_{i + 1}',
        folder='Reforestation_Data',
        fileNamePrefix=f'temp_chunk_{i + 1}_lulc_2020_land_cover',
        fileFormat='CSV'
    )
    task.start()

    print(f"Exporting chunk {i + 1}...")

    while task.active():
        print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
        time.sleep(30)

    print(f'Chunk {i + 1} export done.')

    # Loading the exported CSV for this chunk
    temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_lulc_2020_land_cover.csv'
    temp_chunk_df = pd.read_csv(temp_chunk_path)

    # Appending to combined DataFrame
    combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)

    # Saving combined results to the output CSV
    combined_df.to_csv(output_csv_path, index=False)

    # Removing the temporary file
    if os.path.exists(temp_chunk_path):
        os.remove(temp_chunk_path)
        print(f"Deleted {temp_chunk_path}")

print("All chunks processed, combined results saved to:", output_csv_path)


In [ ]:

# landmask = ee.Image("projects/glad/OceanMask").lte(1)
# landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC').updateMask(landmask)

# # Class codes and names of interest
# classCodes = [208, 240, 248, 245]
# classesOfInterest = ["permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees"]

# maskedLandCover = landCover.remap(classCodes, classCodes, 255)

# def calculate_area(feature):
#     areas = {}


#     for class_code, class_name in zip(classCodes, classesOfInterest):
#         current_class_area = maskedLandCover.eq(class_code) \
#             .multiply(ee.Image.pixelArea()) \
#             .reduceRegion(
#                 reducer=ee.Reducer.sum(),
#                 geometry=feature.geometry(),
#                 scale=30,
#                 maxPixels=1e9
#             )

#         area_value = ee.Number(current_class_area.get('remapped', 0)).divide(1e6)  # Convert to km^2
#         areas[class_name] = area_value

#     return feature.set(areas)

# output_csv_path = '../input/Reforestation_Data/extra_land_cover_results.csv'


# if os.path.exists(output_csv_path):
#     combined_df = pd.read_csv(output_csv_path)
# else:
#     combined_df = pd.DataFrame()


# for i, fc_chunk in enumerate(chunks):
#     print(f"Processing chunk {i + 1}...")


#     area_results_chunk = fc_chunk.map(calculate_area)


#     task = ee.batch.Export.table.toDrive(
#         collection=area_results_chunk,
#         description=f'extra_land_cover_chunk_{i + 1}',
#         folder='Reforestation_Data',
#         fileNamePrefix=f'extra_land_cover_chunk_{i + 1}_with_areas',
#         fileFormat='CSV'
#     )
#     task.start()

#     print(f"Exporting processed chunk {i + 1}...")

#     while task.active():
#         print(f'Polling for task (id: {task.id}) of processed chunk {i + 1}.')
#         time.sleep(30)

#     print(f'Processed chunk {i + 1} export done.')


#     processed_chunk_path = f'../input/Reforestation_Data/extra_land_cover_chunk_{i + 1}_with_areas.csv'
#     processed_chunk_df = pd.read_csv(processed_chunk_path)


#     combined_df = pd.concat([combined_df, processed_chunk_df], ignore_index=True)

#     combined_df.to_csv(output_csv_path, index=False)
#             # Removing the temporary chunk file after processing
#     if os.path.exists(temp_chunk_path):
#         os.remove(temp_chunk_path)
#         print(f"Deleted {temp_chunk_path}")

# print("All processed chunks saved to:", output_csv_path)


##BUILT AREA CHARACTERISTICS EXTRACTION(BUILT_AREA)

In [ ]:


# Output CSV path for built area results
output_csv_path = '../input/Reforestation_Data/merged_built_area_results.csv'

# Check if there's already an existing combined CSV
if os.path.exists(output_csv_path):
    combined_df = pd.read_csv(output_csv_path)
else:
    combined_df = pd.DataFrame()

# Loading the Earth Engine built area image for 2018
builtImage = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").select('built_characteristics')

# calculating the built area for each feature
def calculate_built_area(feature):
    polygon = feature.geometry()
    built_area = builtImage.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon,
        scale=10,
        maxPixels=1e9
    ).get('built_characteristics')

    built_area = ee.Number(built_area).max(0)
    return feature.set('built_area', built_area)

# Looping through chunks to process each chunk's built area
for i, chunk in enumerate(chunks):
    print(f"Processing built area for chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__

    try:
        # Converting the GeoJSON chunk to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the built area calculation function over the FeatureCollection
    built_area_results_chunk = fc_chunk.map(calculate_built_area)

    # Exporting each chunk to Google Drive as CSV
    task = ee.batch.Export.table.toDrive(
        collection=built_area_results_chunk,
        description=f'asia_chunk_built_area_{i + 1}',
        folder='Reforestation_Data',
        fileNamePrefix=f'temp_chunk_{i + 1}_built_area_2018',
        fileFormat='CSV'
    )
    task.start()

    print(f"Exporting built area for chunk {i + 1}...")

    # Polling the task until it's finished
    while task.active():
        print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
        time.sleep(30)

    print(f'Chunk {i + 1} export done.')

    # Loading the exported CSV for this chunk
    temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_built_area_2018.csv'
    temp_chunk_df = pd.read_csv(temp_chunk_path)

    # Appending the current chunk results to the combined DataFrame
    combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)

    # Saving combined results to the output CSV
    combined_df.to_csv(output_csv_path, index=False)

    # Removing the temporary chunk file after processing
    if os.path.exists(temp_chunk_path):
        os.remove(temp_chunk_path)
        print(f"Deleted {temp_chunk_path}")

print("All chunks processed, built area results saved to:", output_csv_path)


In [ ]:



# output_csv_path = '../input/Reforestation_Data/merged_built_area_results.csv'


# if os.path.exists(output_csv_path):
#     combined_df = pd.read_csv(output_csv_path)
# else:
#     combined_df = pd.DataFrame()

# builtImage = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").select('built_characteristics')

# def calculate_built_area(feature):
#     polygon = feature.geometry()
#     built_area = builtImage.reduceRegion(
#         reducer=ee.Reducer.sum(),
#         geometry=polygon,
#         scale=10,
#         maxPixels=1e9
#     ).get('built_characteristics')

#     built_area = ee.Number(built_area).max(0)
#     return feature.set('built_area', built_area)


# for i, chunk in enumerate(chunks):
#     print(f"Processing built area for chunk {i + 1}/{len(chunks)}")

#     gdf_json_chunk = chunk.__geo_interface__

#     fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
#     built_area_results_chunk = fc_chunk.map(calculate_built_area)
#     task = ee.batch.Export.table.toDrive(
#         collection=built_area_results_chunk,
#         description=f'asia_chunk_built_area_{i + 1}',
#         folder='Reforestation_Data',
#         fileNamePrefix=f'temp_chunk_{i + 1}_built_area_2018',
#         fileFormat='CSV'
#     )
#     task.start()

#     print(f"Exporting built area for chunk {i + 1}...")

#     while task.active():
#         print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
#         time.sleep(30)

#     print(f'Chunk {i + 1} export done.')

#     temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_built_area_2018.csv'
#     temp_chunk_df = pd.read_csv(temp_chunk_path)

#     combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)

#     combined_df.to_csv(output_csv_path, index=False)
#             # Removing the temporary chunk file after processing
#     if os.path.exists(temp_chunk_path):
#         os.remove(temp_chunk_path)
#         print(f"Deleted {temp_chunk_path}")

# print("All chunks processed, built area results saved to:", output_csv_path)


##ROAD NETWORKS COLUMNS(total_road_length_km)

In [ ]:



output_csv_path = '../input/Reforestation_Data/merged_road_length_results.csv'

# Checing if there's already an existing combined CSV
if os.path.exists(output_csv_path):
    combined_df = pd.read_csv(output_csv_path)
else:
    combined_df = pd.DataFrame()

# Loading the Earth Engine road datasets
roadsAfrica = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-africa')
roadsAmericas = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-americas')
roadsAsia = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-asia')
roadsEurope = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-europe')
roadsOceaniaEast = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-east')
roadsOceaniaWest = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-west')

# Merging all road datasets into a single collection
roads = roadsAfrica.merge(roadsAmericas).merge(roadsAsia).merge(roadsEurope).merge(roadsOceaniaEast).merge(roadsOceaniaWest)

#  Calculating road length for each feature (polygon)
def calculate_road_length(polygon):
    # Finding all roads that intersect the polygon
    intersecting_roads = roads.filterBounds(polygon.geometry())

    #  clipping road geometry and calculating its length within the polygon
    def clip_and_calculate_length(road):
        road_geom = road.geometry()
        polygon_geom = polygon.geometry()
        clipped = road_geom.intersection(polygon_geom, ee.ErrorMargin(1))
        return ee.Feature(clipped).set('length', clipped.length())

    # Mapping the clipping and length calculation over the intersecting roads
    clipped_roads = intersecting_roads.map(clip_and_calculate_length)

    # Summing the total road length in the polygon
    road_length_sum = clipped_roads.reduceColumns(
        reducer=ee.Reducer.sum(),
        selectors=['length']
    ).get('sum')

    # Counting the number of intersecting roads
    intersecting_roads_count = intersecting_roads.size()

    # ting total road length (in km) and road count as properties on the polygon
    return polygon.set({
        'total_road_length_km': ee.Number(road_length_sum).divide(1000),
        'intersecting_roads_count': intersecting_roads_count
    })

# Looping through chunks to process road lengths for each chunk
for i, chunk in enumerate(chunks):
    print(f"Processing road length for chunk {i + 1}/{len(chunks)}")

    # Converting the current GeoDataFrame chunk to GeoJSON
    gdf_json_chunk = chunk.__geo_interface__

    try:
        # Converting the GeoJSON chunk to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the road length calculation function over the FeatureCollection
    road_length_results_chunk = fc_chunk.map(calculate_road_length)

    # Exporting each chunk to Google Drive as CSV
    task = ee.batch.Export.table.toDrive(
        collection=road_length_results_chunk,
        description=f'chunk_road_length_{i + 1}',
        folder='Reforestation_Data',
        fileNamePrefix=f'temp_chunk_{i + 1}_road_length',
        fileFormat='CSV'
    )
    task.start()

    print(f"Exporting road length for chunk {i + 1}...")

    # Polling the task until it's finished
    while task.active():
        print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
        time.sleep(30)

    print(f'Chunk {i + 1} export done.')

    # Loading the exported CSV for this chunk
    temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_road_length.csv'
    temp_chunk_df = pd.read_csv(temp_chunk_path)

    # Appending the current chunk results to the combined DataFrame
    combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)

    # Saving combined results to the output CSV
    combined_df.to_csv(output_csv_path, index=False)

    # Removing the temporary chunk file after processing
    if os.path.exists(temp_chunk_path):
        os.remove(temp_chunk_path)
        print(f"Deleted {temp_chunk_path}")

print("All chunks processed, road length results saved to:", output_csv_path)


In [ ]:

# output_csv_path = '../input/Reforestation_Data/merged_road_length_results.csv'

# if os.path.exists(output_csv_path):
#     combined_df = pd.read_csv(output_csv_path)
# else:
#     combined_df = pd.DataFrame()

# roadsAfrica = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-africa')
# roadsAmericas = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-americas')
# roadsAsia = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-asia')
# roadsEurope = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-europe')
# roadsOceaniaEast = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-east')
# roadsOceaniaWest = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-west')

# roads = roadsAfrica.merge(roadsAmericas).merge(roadsAsia).merge(roadsEurope).merge(roadsOceaniaEast).merge(roadsOceaniaWest)


# def calculate_road_length(polygon):

#     intersecting_roads = roads.filterBounds(polygon.geometry())

#     def clip_and_calculate_length(road):
#         road_geom = road.geometry()
#         polygon_geom = polygon.geometry()
#         clipped = road_geom.intersection(polygon_geom, ee.ErrorMargin(1))
#         return ee.Feature(clipped).set('length', clipped.length())

#     clipped_roads = intersecting_roads.map(clip_and_calculate_length)


#     road_length_sum = clipped_roads.reduceColumns(
#         reducer=ee.Reducer.sum(),
#         selectors=['length']
#     ).get('sum')


#     intersecting_roads_count = intersecting_roads.size()


#     return polygon.set({
#         'total_road_length_km': ee.Number(road_length_sum).divide(1000),
#         'intersecting_roads_count': intersecting_roads_count
#     })


# for i, chunk in enumerate(chunks):
#     print(f"Processing road length for chunk {i + 1}/{len(chunks)}")


#     gdf_json_chunk = chunk.__geo_interface__


#     fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)


#     road_length_results_chunk = fc_chunk.map(calculate_road_length)


#     task = ee.batch.Export.table.toDrive(
#         collection=road_length_results_chunk,
#         description=f'asia_chunk_road_length_{i + 1}',
#         folder='Reforestation_Data',
#         fileNamePrefix=f'temp_chunk_{i + 1}_road_length',
#         fileFormat='CSV'
#     )
#     task.start()

#     print(f"Exporting road length for chunk {i + 1}...")


#     while task.active():
#         print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
#         time.sleep(30)

#     print(f'Chunk {i + 1} export done.')


#     temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_road_length.csv'
#     temp_chunk_df = pd.read_csv(temp_chunk_path)


#     combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)


#     combined_df.to_csv(output_csv_path, index=False)
#             # Removing the temporary chunk file after processing
#     if os.path.exists(temp_chunk_path):
#         os.remove(temp_chunk_path)
#         print(f"Deleted {temp_chunk_path}")

# print("All chunks processed, road length results saved to:", output_csv_path)


##GLOBAL FOREST LOSS GROUPS EXTRACTION,using the python code in the "data_analysis.ipny" create the colun s "loss_pre_5","loss_post_3",loss_post_5"

In [ ]:

output_csv_path = '../input/Reforestation_Data/merged_forest_loss_results.csv'

# Checking if there's already an existing combined CSV
if os.path.exists(output_csv_path):
    combined_df = pd.read_csv(output_csv_path)
else:
    combined_df = pd.DataFrame()

# Loading the Global Forest Change 2023 dataset
gfc2017 = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

# Calculate forest loss within each feature (polygon)
def calculate_loss(feature):
    # Selecting the forest loss band and calculate loss area
    loss_image = gfc2017.select(['loss'])
    loss_area_image = loss_image.multiply(ee.Image.pixelArea())
    loss_year = gfc2017.select(['lossyear'])

    # Calculating forest loss area by year within the feature geometry
    loss_by_year = loss_area_image.addBands(loss_year).reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    )

   
    return feature.set(loss_by_year)

# Looping through chunks to process forest loss for each chunk
for i, chunk in enumerate(chunks):
    print(f"Processing forest loss for chunk {i + 1}/{len(chunks)}")

    # Converting the current GeoDataFrame chunk to GeoJSON
    gdf_json_chunk = chunk.__geo_interface__

    try:
        # Converting GeoJSON chunk to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the forest loss calculation function over the FeatureCollection
    loss_results_chunk = fc_chunk.map(calculate_loss)

    # Exporting each chunk's results to Google Drive as CSV
    task = ee.batch.Export.table.toDrive(
        collection=loss_results_chunk,
        description=f'forest_loss_chunk_{i + 1}',
        folder='Reforestation_Data',
        fileNamePrefix=f'temp_chunk_{i + 1}_forest_loss',
        fileFormat='CSV'
    )
    task.start()

    print(f"Exporting forest loss for chunk {i + 1}...")

    # Polling the task until it is completed
    while task.active():
        print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
        time.sleep(30)

    print(f'Chunk {i + 1} export done.')

    # Loading the exported CSV for this chunk
    temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_forest_loss.csv'
    temp_chunk_df = pd.read_csv(temp_chunk_path)

    # Appending the current chunk's results to the combined DataFrame
    combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)

    # Saving the combined results to the output CSV
    combined_df.to_csv(output_csv_path, index=False)

    # Removing the temporary chunk file after processing
    if os.path.exists(temp_chunk_path):
        os.remove(temp_chunk_path)
        print(f"Deleted {temp_chunk_path}")

print("All chunks processed, forest loss results saved to:", output_csv_path)


In [ ]:

# gfc2017 = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')


# output_csv_path = '../input/Reforestation_Data/merged_forest_loss_results.csv'


# if os.path.exists(output_csv_path):
#     combined_df = pd.read_csv(output_csv_path)
# else:
#     combined_df = pd.DataFrame()


# def calculate_loss(feature):
#     loss_image = gfc2017.select(['loss'])
#     loss_area_image = loss_image.multiply(ee.Image.pixelArea())
#     loss_year = gfc2017.select(['lossyear'])

#     loss_by_year = loss_area_image.addBands(loss_year).reduceRegion(
#         reducer=ee.Reducer.sum().group(groupField=1),
#         geometry=feature.geometry(),
#         scale=30,
#         maxPixels=1e9
#     )

#     return feature.set(loss_by_year)

# for i, chunk in enumerate(chunks):
#     print(f"Processing forest loss for chunk {i + 1}/{len(chunks)}")


#     gdf_json_chunk = chunk.__geo_interface__


#     fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)


#     loss_results_chunk = fc_chunk.map(calculate_loss)


#     task = ee.batch.Export.table.toDrive(
#         collection=loss_results_chunk,
#         description=f'forest_loss_chunk_{i + 1}',
#         folder='Reforestation_Data',
#         fileNamePrefix=f'temp_chunk_{i + 1}_forest_loss',
#         fileFormat='CSV'
#     )
#     task.start()

#     print(f"Exporting forest loss for chunk {i + 1}...")


#     while task.active():
#         print(f'Polling for task (id: {task.id}) of chunk {i + 1}.')
#         time.sleep(30)

#     print(f'Chunk {i + 1} export done.')

#     temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{i + 1}_forest_loss.csv'
#     temp_chunk_df = pd.read_csv(temp_chunk_path)


#     combined_df = pd.concat([combined_df, temp_chunk_df], ignore_index=True)


#     combined_df.to_csv(output_csv_path, index=False)
#             # Removing the temporary chunk file after processing
#     if os.path.exists(temp_chunk_path):
#         os.remove(temp_chunk_path)
#         print(f"Deleted {temp_chunk_path}")

# print("All chunks processed, forest loss results saved to:", output_csv_path)


#### Generating the loss columns

In [ ]:

df_km=pd.read_csv('../input/Reforestation_Data/merged_forest_loss_results.csv')

df_km["groups"]

,groups
0,[]
1,[]
2,[]
3,[]
4,[]
...,...
1071,"[{group=15, sum=29764.3157698089}, {group=16, ..."
1072,"[{group=7, sum=5644.555928548177}, {group=10, ..."
1073,"[{group=3, sum=5151.752848307291}, {group=6, s..."
1074,"[{group=15, sum=793.3899449965534}, {group=16,..."


In [ ]:
df_km.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  1076 non-null   object 
 1   country       1076 non-null   object 
 2   descriptio    1076 non-null   object 
 3   groups        1076 non-null   object 
 4   planting_d    873 non-null    object 
 5   project_id    1076 non-null   object 
 6   site_id_re    1076 non-null   object 
 7   site_sqkm     1076 non-null   float64
 8   survival_r    887 non-null    float64
 9   trees_plan    1076 non-null   float64
 10  .geo          1076 non-null   object 
dtypes: float64(3), object(8)
memory usage: 92.6+ KB


In [ ]:
import ast


def parse_loss_groups(loss_groups):

    all_groups = {f"group_{i}": 0 for i in range(24)}  # defining the 24 groups (0 to 23)-2000 to 2023

    if not loss_groups or loss_groups == '[]':
        return all_groups

    loss_groups = loss_groups.replace('group=', '"group":').replace('sum=', '"sum":')
    parsed_groups = ast.literal_eval(loss_groups)


    for item in parsed_groups:
        all_groups[f"group_{item['group']}"] = item['sum']

    return all_groups


expanded_columns = df_km['groups'].apply(parse_loss_groups)


expanded_df = pd.DataFrame(expanded_columns.tolist())


group_mapping = {
    0: '2000',
    1: '2001',
    2: '2002',
    3: '2003',
    4: '2004',
    5: '2005',
    6: '2006',
    7: '2007',
    8: '2008',
    9: '2009',
    10: '2010',
    11: '2011',
    12: '2012',
    13: '2013',
    14: '2014',
    15: '2015',
    16: '2016',
    17: '2017',
    18: '2018',
    19: '2019',
    20: '2020',
    21: '2021',
    22: '2022',
    23: '2023'
}


expanded_df.rename(columns=lambda x: group_mapping.get(int(x.split('_')[1]), x), inplace=True)


df_km = df_km.join(expanded_df)

In [ ]:
#  When only year is given in planting_date
df_km["planting_date_reported"]=df_km["planting_d"]
# def calculate_loss_pre_5(row):
#     planted_year = row['planting_date_reported']
#     years = [str(planted_year - i) for i in range(1, 6)]
#     losses = [row[year] for year in years if year in df_km.columns]
#     return np.nanmean(losses) if losses else np.nan

# def calculate_loss_post_3(row):
#     planted_year = row['planting_date_reported']
#     years = [str(planted_year + i) for i in range(1, 4)]
#     losses = [row[year] for year in years if year in df_km.columns]
#     return np.nanmean(losses) if losses else np.nan
# def calculate_loss_post_5(row):
#     planted_year = row['planting_date_reported']
#     years = [str(planted_year + i) for i in range(1, 6)]
#     losses = [row[year] for year in years if year in df_km.columns]
#     return np.nanmean(losses) if losses else np.nan

# df_km['loss_pre_5'] = df_km.apply(calculate_loss_pre_5, axis=1)
# df_km['loss_post_3'] = df_km.apply(calculate_loss_post_3, axis=1)
# df_km['loss_post_5'] = df_km.apply(calculate_loss_post_5, axis=1)



# df_km[['planting_date_reported', 'loss_pre_5','loss_post_3']].tail()

In [ ]:
# if year not given and have to extract
import numpy as np

def calculate_loss_pre_5(row):
    try:
        planted_date = pd.to_datetime(row['planting_date_reported'], errors='coerce')
        if pd.isnull(planted_date):
            return np.nan
        planted_year = planted_date.year
        years = [str(planted_year - i) for i in range(1, 6)]
        losses = [row[year] for year in years if year in df_km.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan

def calculate_loss_post_3(row):
    try:
        planted_date = pd.to_datetime(row['planting_date_reported'], errors='coerce')
        if pd.isnull(planted_date):
            return np.nan
        planted_year = planted_date.year
        years = [str(planted_year + i) for i in range(1, 4)]
        losses = [row[year] for year in years if year in df_km.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan

def calculate_loss_post_5(row):
    try:
        planted_date = pd.to_datetime(row['planting_date_reported'], errors='coerce')
        if pd.isnull(planted_date):
            return np.nan
        planted_year = planted_date.year
        years = [str(planted_year + i) for i in range(1, 6)]
        losses = [row[year] for year in years if year in df_km.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan


df_km['loss_pre_5'] = df_km.apply(calculate_loss_pre_5, axis=1)
df_km['loss_post_3'] = df_km.apply(calculate_loss_post_3, axis=1)
df_km['loss_post_5'] = df_km.apply(calculate_loss_post_5, axis=1)

In [ ]:
columns_to_drop = [str(year) for year in range(2000, 2024)] + ['groups']
df_km = df_km.drop(columns=columns_to_drop, errors='ignore')
df_km.tail()

,system:index,country,descriptio,planting_d,project_id,site_id_re,site_sqkm,survival_r,trees_plan,.geo,planting_date_reported,loss_pre_5,loss_post_3,loss_post_5
1071,000000000000000000bc,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24315018...",NaN,NaN,NaN,NaN
1072,000000000000000000bd,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24283809...",NaN,NaN,NaN,NaN
1073,000000000000000000be,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.25306721...",NaN,NaN,NaN,NaN
1074,000000000000000000bf,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24170103...",NaN,NaN,NaN,NaN
1075,000000000000000000c0,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.37896101...",NaN,NaN,NaN,NaN


In [ ]:
df_km.to_csv("'../input/Reforestation_Data/plant_forest_loss_by_polygon.csv")


##NDVI per MONTH

In [ ]:


# Sentinel-2 dataset uploading
S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

# Calculating NDVI for each Sentinel-2 image
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Reducing the image collection to monthly mean NDVI
def reduce_to_monthly(image_collection):
    monthly = image_collection.select('NDVI').mean().rename('NDVI')
    return monthly

# Calculating and exporting monthly NDVI for each chunk
def calculate_and_export_monthly_ndvi(month):
    combined_monthly_data = pd.DataFrame()  

    for chunk_index, chunk in enumerate(chunks):
        print(f"Processing chunk {chunk_index + 1}/{len(chunks)} for month {month}...")

        # Converting the current GeoDataFrame chunk to GeoJSON format
        gdf_json_chunk = chunk.__geo_interface__
        
        # Converting the GeoJSON chunk to an Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)

        # Filtering Sentinel-2 images by the specified month and calculate NDVI
        monthly_s2 = S2.filter(ee.Filter.calendarRange(month, month, 'month')).map(calculate_ndvi)
        
        # Reducing the collection to a single NDVI image for the month
        monthly_ndvi = reduce_to_monthly(monthly_s2)

        # Calculating the mean NDVI for each feature (polygon) in the chunk
        mean_ndvi = monthly_ndvi.reduceRegions(
            collection=fc_chunk,
            reducer=ee.Reducer.mean(),
            scale=30
        )

        # Exporting the NDVI data to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=mean_ndvi,
            folder='Reforestation_Data',
            description=f'temp_chunk_{chunk_index + 1}_month_{month}',
            fileFormat='CSV'
        )
        task.start()

        # Monitoring the export task until it completes
        while task.active():
            print(f'Waiting for task (id: {task.id}) to complete for chunk {chunk_index + 1}, month {month}...')
            time.sleep(30)

        print(f'Chunk {chunk_index + 1}, month {month} export done.')

        # Loading the exported CSV for the chunk and append it to the combined DataFrame
        temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{chunk_index + 1}_month_{month}.csv'
        if os.path.exists(temp_chunk_path):
            temp_chunk_df = pd.read_csv(temp_chunk_path)
            combined_monthly_data = pd.concat([combined_monthly_data, temp_chunk_df], ignore_index=True)

           
            os.remove(temp_chunk_path)
            print(f"Deleted {temp_chunk_path}")

    # Saving the combined monthly data to a CSV file
    output_monthly_path = f'../input/Reforestation_Data/monthly_combined_ndvi_month_{month}.csv'
    combined_monthly_data.to_csv(output_monthly_path, index=False)
    print(f"Combined data for month {month} saved to {output_monthly_path}")

# Looping through each month (1 to 12) to calculate and export NDVI
for month in range(1, 13):
    calculate_and_export_monthly_ndvi(month)
    print(f'Exporting combined NDVI for month {month}...')

print('All months processed and combined NDVI files exported.')


In [ ]:


# # Calculating NDVI
# def calculate_ndvi(image):
#     ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
#     return image.addBands(ndvi)


# def reduce_to_monthly(image_collection):
#     monthly = image_collection.select('NDVI').mean().rename('NDVI')
#     return monthly

# # Processing each month and  combining all chunks
# def calculate_and_export_monthly_ndvi(month):
#     combined_monthly_data = pd.DataFrame()

#     for chunk_index, chunk in enumerate(chunks):
#         print(f"Processing chunk {chunk_index + 1}/{len(chunks)} for month {month}...")

#         gdf_json_chunk = chunk.__geo_interface__
#         fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)

#         # Filtering Sentinel-2 images by month
#         monthly_s2 = S2.filter(ee.Filter.calendarRange(month, month, 'month')).map(calculate_ndvi)
#         monthly_ndvi = reduce_to_monthly(monthly_s2)

#         # Calculating mean NDVI over each chunk
#         mean_ndvi = monthly_ndvi.reduceRegions(
#             collection=fc_chunk,
#             reducer=ee.Reducer.mean(),
#             scale=30
#         )


#         task = ee.batch.Export.table.toDrive(
#             collection=mean_ndvi,
#             folder='Reforestation_Data',
#             description=f'temp_chunk_{chunk_index + 1}_month_{month}',
#             fileFormat='CSV'
#         )
#         task.start()

#         # Just monitoring progress
#         while task.active():
#             print(f'Waiting for task (id: {task.id}) to complete for chunk {chunk_index + 1}, month {month}...')
#             time.sleep(30)

#         print(f'Chunk {chunk_index + 1}, month {month} export done.')

#         # Loading the exported CSV for the chunks and appending it to the combined DataFrame
#         temp_chunk_path = f'../input/Reforestation_Data/temp_chunk_{chunk_index + 1}_month_{month}.csv'
#         temp_chunk_df = pd.read_csv(temp_chunk_path)

#         combined_monthly_data = pd.concat([combined_monthly_data, temp_chunk_df], ignore_index=True)

#     # Saving the  combined monthly data to a CSV file
#     output_monthly_path = f'../input/Reforestation_Data/monthly_combined_ndvi_month_{month}.csv'
#     combined_monthly_data.to_csv(output_monthly_path, index=False)
#     print(f"Combined data for month {month} saved to {output_monthly_path}")
#             # Removing the temporary chunk file after processing
#     if os.path.exists(temp_chunk_path):
#         os.remove(temp_chunk_path)
#         print(f"Deleted {temp_chunk_path}")


# for month in range(1, 13):
#     calculate_and_export_monthly_ndvi(month)
#     print(f'Exporting combined NDVI for month {month}...')

# print('All months processed and combined NDVI files exported.')



/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Exporting month 1...
Exporting month 2...
Exporting month 3...
Exporting month 4...
Exporting month 5...
Exporting month 6...
Exporting month 7...
Exporting month 8...
Exporting month 9...
Exporting month 10...
Exporting month 11...
Exporting month 12...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiti

In [ ]:



data_dir = f'../input/Reforestation_Data/'


combined_df = pd.DataFrame()


for month in range(1, 13):

    file_path = os.path.join(data_dir, f'monthly_combined_ndvi_month_{month}.csv')

    df = pd.read_csv(file_path)


    ndvi_column_name = f'NDVI_0{month}'
    df.rename(columns={'mean': ndvi_column_name}, inplace=True)

    if combined_df.empty:

        combined_df = df
    else:

        combined_df = pd.merge(combined_df, df, on=list(df.columns.difference([ndvi_column_name])), how='outer')


output_file = os.path.join(data_dir, 'combined_plant_monthly_ndvi.csv')
combined_df.to_csv(output_file, index=False)

print(f'Combined data saved to {output_file}')


Combined data saved to /content/drive/MyDrive/Plant_planet/combined_plant_monthly_ndvi.csv


In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  1076 non-null   object 
 1   country       1076 non-null   object 
 2   descriptio    1076 non-null   object 
 3   NDVI_01       1074 non-null   float64
 4   planting_d    873 non-null    object 
 5   project_id    1076 non-null   object 
 6   site_id_re    1076 non-null   object 
 7   site_sqkm     1076 non-null   float64
 8   survival_r    887 non-null    float64
 9   trees_plan    1076 non-null   float64
 10  .geo          1076 non-null   object 
 11  NDVI_02       1074 non-null   float64
 12  NDVI_03       1074 non-null   float64
 13  NDVI_04       1074 non-null   float64
 14  NDVI_05       1074 non-null   float64
 15  NDVI_06       1074 non-null   float64
 16  NDVI_07       1074 non-null   float64
 17  NDVI_08       1074 non-null   float64
 18  NDVI_09       1074 non-null 

In [ ]:
df=combined_df
def find_top_three_ndvi_months(row):
    ndvi_columns = ['NDVI_01', 'NDVI_02', 'NDVI_03', 'NDVI_04', 'NDVI_05', 'NDVI_06',
                    'NDVI_07', 'NDVI_08', 'NDVI_09', 'NDVI_010', 'NDVI_011', 'NDVI_012']
    ndvi_values = []

    # Collecting NDVI values and their corresponding months
    for column in ndvi_columns:
        month_number = column.split('_')[1]
        ndvi_value = row[column]
        ndvi_values.append((ndvi_value, month_number))

    # Sorting the list by NDVI values in descending order
    ndvi_values.sort(reverse=True, key=lambda x: x[0])

    # Extracting the top three months
    top_three_months = [int(month) for _, month in ndvi_values[:3]]

    return top_three_months


df['Top_Three_NDVI_Months'] = df.apply(find_top_three_ndvi_months, axis=1)


ndvi_columns_to_drop = ['NDVI_01', 'NDVI_02', 'NDVI_03', 'NDVI_04', 'NDVI_05', 'NDVI_06',
                        'NDVI_07', 'NDVI_08', 'NDVI_09', 'NDVI_010', 'NDVI_011', 'NDVI_012']


df.drop(columns=ndvi_columns_to_drop, inplace=True)


df.tail()

,system:index,country,descriptio,planting_d,project_id,site_id_re,site_sqkm,survival_r,trees_plan,.geo,Top_Three_NDVI_Months
1071,000000000000000000bc,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""GeometryCollection"",""geometries"":[{""t...","[10, 3, 1]"
1072,000000000000000000bd,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24284252...","[10, 1, 7]"
1073,000000000000000000be,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.25322336...","[10, 2, 3]"
1074,000000000000000000bf,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24181244...","[10, 1, 2]"
1075,000000000000000000c0,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.37923753...","[6, 1, 12]"


In [ ]:
df.to_csv('../input/Reforestation_Data/combined_plant_monthly_ndvi.csv', index=False)

*** Conclusion ***
Manually or through code  combine the unique columns to have complete dataset for each reforestation organization

In [ ]:
import os
import pandas as pd

dataframes = []
all_columns = set()


folder_path = '../input/Reforestation_Data'

for file_name in os.listdir(folder_path):

    if file_name.endswith('.csv') and not file_name.startswith('monthly_combined_ndvi_month_'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
        all_columns.update(df.columns)


common_columns = list(set.intersection(*[set(df.columns) for df in dataframes]))


merged_df = dataframes[0][common_columns].copy()


unique_columns_first_df = list(set(dataframes[0].columns) - set(common_columns))
merged_df = pd.concat([merged_df, dataframes[0][unique_columns_first_df]], axis=1)


for df in dataframes[1:]:

    unique_columns = list(set(df.columns) - set(common_columns))

    merged_df = pd.concat([merged_df, df[unique_columns]], axis=1)


columns_to_drop = ["system:index", "planting_d", "Unnamed: 0"]
merged_df = merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns])


if 'site_id_re_x' in merged_df.columns and 'site_id_re_y' in merged_df.columns:
    merged_df['site_id_re'] = merged_df['site_id_re_x'].combine_first(merged_df['site_id_re_y'])
    merged_df = merged_df.drop(columns=['site_id_re_x', 'site_id_re_y'])

# merged_df['host_name'] = 'plant for plant'


# merged_df['url'] = 'https://web.plant-for-the-planet.org/en/sirpnigeria-org?site=site_bKgRAzMysTSivm1'

output_file_path = os.path.join(folder_path, 'Updated_Reforestation_Data.csv')
merged_df.to_csv(output_file_path, index=False)

print(f"Merged CSV saved to {output_file_path}")


Merged CSV saved to /content/drive/MyDrive/Plant_planet/plant_planet.csv
